In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
# finding years on which data has to be processed, below is the logic which filters based on the file date then selects distinct years, collect is used to collect output as a list

race_results_list = spark.read.format("delta").load(f"{presentation_container_folder_path}/race_results")\
.filter(f"file_date = '{v_file_date}'")\
.select("race_year")\
.distinct()\
.collect()

# getting the distinct years from the list

race_year_list = []
for race_year in race_results_list:
    race_year_list.append(race_year.race_year)

In [0]:
# finding years on which data has to be processed, below is the logic which filters based on the file date then selects distinct years, collect is used to collect output as a list

race_results_list = spark.read.format("delta").load(f"{presentation_container_folder_path}/race_results")\
.filter(f"file_date = '{v_file_date}'")\
.select("race_year")\
.distinct()\
.collect()

# getting the distinct years from the list

race_year_list = []
for race_year in race_results_list:
    race_year_list.append(race_year.race_year)

In [0]:
display(race_results_df.orderBy(col("race_year").desc()))

In [0]:
from pyspark.sql.functions import *
driver_standings_df = race_results_df\
    .groupBy("race_year", "driver_name", "driver_nationality")\
        .agg(sum("points").alias("total_points"), count(when(col("position")== 1, True)).alias("wins"))\
            .orderBy(col("race_year"),col("total_points").desc(), col("wins").desc())

In [0]:
from pyspark.sql.window import Window
window_spec = Window.partitionBy("race_year").orderBy(col("total_points").desc(), col("wins").desc())
final_df = driver_standings_df.withColumn("rank", rank().over(window_spec))

In [0]:
#final_df.write.mode("overwrite").parquet(f"{presentation_container_folder_path}/driver_standings")

#final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation_db.driver_standings")

# load_incremental_data(final_df, "race_year", "f1_presentation_db", "driver_standings")

merge_condition = "target_tab.driver_name = source_tab.driver_name AND target_tab.race_year = source_tab.race_year"

merge_incremental_data(final_df, presentation_container_folder_path, merge_condition, "race_year", "f1_presentation_db", "driver_standings")